In [59]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

start_time = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)
    
DCBS_DF = DCBS_DataFrame
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

B_DF = DTBS_DataFrame['B']

# 运行这个文件之前必须先去运行一下DCBS_toDataFrame文件，这样跑出来的结果才是最新的
# 计算所有转债每天的涨跌幅，并找出涨幅大于前N个交易日的q分位数涨跌幅的均值的转债，用于每天2点选出符合每日热门+双低策略的转债

In [60]:
td_A_DF = A_DF[A_DF['date'] == '2023-08-31'].copy()
temp_df = pd.merge(td_A_DF, B_DF[['bond_code','cn','sc','sn','cat1','cat2']], on='bond_code',how='left')
temp_df

,bond_code,date,cpr,dp,bl,trt,ytm,yl,dl,ia,...,qs15,qs30,xx15,xx30,hs,cn,sc,sn,cat1,cat2
0,110088.SH,2023-08-31,43.01,118.800,30.000,1.34,-1.36,5.044,161.810,1,...,0.0,0.0,13.0,13.0,0.0,淮22转债,600985.SH,淮北矿业,煤炭,煤炭开采
1,113545.SH,2023-08-31,42.59,117.970,9.865,0.59,-1.92,2.123,160.560,1,...,0.0,0.0,3.0,3.0,0.0,金能转债,603113.SH,金能科技,煤炭,焦炭Ⅱ
2,113631.SH,2023-08-31,14.67,124.378,9.299,2.32,-2.03,4.192,139.048,1,...,0.0,0.0,0.0,0.0,0.0,皖天转债,603689.SH,皖天然气,公用事业,燃气Ⅱ
3,123128.SZ,2023-08-31,59.39,104.690,13.791,1.75,2.28,4.173,164.080,1,...,0.0,0.0,15.0,30.0,0.0,首华转债,300483.SZ,首华燃气,公用事业,燃气Ⅱ
4,123156.SZ,2023-08-31,21.98,124.720,3.970,1.99,-0.64,4.964,146.700,1,...,0.0,0.0,0.0,0.0,0.0,博汇转债,300839.SZ,博汇股份,石油石化,炼化及贸易
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,123215.SZ,2023-08-31,50.21,119.234,10.000,2.63,0.58,5.926,169.444,1,...,0.0,0.0,7.0,7.0,0.0,铭利转债,301268.SZ,铭利达,NaN,NaN
526,123217.SZ,2023-08-31,39.02,129.900,5.700,16.61,-2.09,5.940,168.920,1,...,0.0,0.0,0.0,0.0,0.0,富仕转债,300852.SZ,四会富仕,NaN,NaN
527,113068.SH,2023-08-31,24.00,119.955,14.500,5.79,-1.10,5.910,143.955,1,...,0.0,0.0,0.0,0.0,0.0,N金铜转,601609.SH,金田股份,NaN,NaN
528,123219.SZ,2023-08-31,39.31,127.647,6.000,66.85,-1.22,5.948,166.957,1,...,0.0,0.0,0.0,0.0,0.0,宇瞳转债,300790.SZ,宇瞳光学,NaN,NaN


In [62]:
print(list(temp_df[temp_df['cat2']=='消费电子']['bond_code']))

['113045.SH', '113577.SH', '123100.SZ', '123143.SZ', '123153.SZ', '128118.SZ', '128136.SZ', '113667.SH', '123193.SZ', '113672.SH', '127090.SZ']


In [54]:
# DataFrame新增列用于存放前一天收盘价、涨跌幅
# print(DCBS_DF.info())
DCBS_DF.sort_values(by = ['bond_code','date'], inplace=True, ascending=[True,True])
DCBS_DF['lastday_dp'] = DCBS_DF.groupby('bond_code')['dp'].shift(1)
DCBS_DF['change'] = DCBS_DF['dp'] / DCBS_DF['lastday_dp'] - 1
print(DCBS_DF)

A_DF.sort_values(by = ['bond_code','date'], inplace=True, ascending=[True,True])
A_DF['lastday_dp'] = A_DF.groupby('bond_code')['dp'].shift(1)
A_DF['change'] = A_DF['dp'] / A_DF['lastday_dp'] - 1
A_DF = A_DF[A_DF['date']>='2023-04-27'].copy()
print(A_DF)

                      bond_code        date    cpr       dp       dl    ytm  \
id                                                                            
110043.SH,2023-04-27  110043.SH  2023-04-27  11.78  112.976  124.756  -8.10   
110043.SH,2023-04-28  110043.SH  2023-04-28  12.25  113.247  125.497  -8.43   
110043.SH,2023-05-04  110043.SH  2023-05-04  11.49  114.459  125.949  -9.96   
110043.SH,2023-05-05  110043.SH  2023-05-05  11.91  114.298  126.208  -9.81   
110043.SH,2023-05-08  110043.SH  2023-05-08  11.45  116.600  128.050 -12.42   
...                         ...         ...    ...      ...      ...    ...   
128144.SZ,2023-08-25  128144.SZ  2023-08-25  55.37  115.365  170.735  -0.52   
128144.SZ,2023-08-28  128144.SZ  2023-08-28  53.40  116.004  169.404  -0.68   
128144.SZ,2023-08-29  128144.SZ  2023-08-29  52.88  116.435  169.315  -0.79   
128144.SZ,2023-08-30  128144.SZ  2023-08-30  51.26  116.030  167.290  -0.69   
128144.SZ,2023-08-31  128144.SZ  2023-08-31  50.61  

In [55]:
print(sorted(list(set(list(DCBS_DF['date'])))))
print(sorted(list(set(list(A_DF['date'])))))

['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25', '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10', '2023

In [56]:
DCBS_DF['当日是否属于热门转债'] = (DCBS_DF['change'] >= 0.035).astype(int)

m = 2
n = 63

DCBS_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = DCBS_DF.groupby('bond_code')
DCBS_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
DCBS_DF['是否属于过去一段时间内的热门转债'] = (DCBS_DF['过去一段时间是热门转债的天数'] >= m).astype(int)

# 这里的功能是找出前n天所有热门的日期，这个步骤会耗费大量运算资源，非必要不输出这一条信息
DCBS_DF['last_n_date'] = DCBS_DF.groupby('bond_code')['date'].shift(n)

def filter_dates(group):
    group['前n天内所有热门的日期'] = group.apply(
        lambda row: '、'.join(group[
            (group['当日是否属于热门转债'] == 1)&
            (group['date'] >= row['last_n_date'])& 
            (group['date'] < row['date'])              
        ]['date'].tolist()),
        axis=1
    )
    return group

DCBS_DF['前n天内所有热门的日期'] = DCBS_DF.groupby('bond_code', group_keys=True).apply(filter_dates).reset_index(level=0, drop=True)['前n天内所有热门的日期']
print(DCBS_DF)



A_DF['当日是否属于热门转债'] = (A_DF['change'] >= 0.035).astype(int)
A_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = A_DF.groupby('bond_code')
A_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
A_DF['是否属于过去一段时间内的热门转债'] = (A_DF['过去一段时间是热门转债的天数'] >= m).astype(int)

# 这里的功能是找出前n天所有热门的日期，这个步骤会耗费大量运算资源，非必要不输出这一条信息
A_DF['last_n_date'] = A_DF.groupby('bond_code')['date'].shift(n)

def filter_dates(group):
    group['前n天内所有热门的日期'] = group.apply(
        lambda row: '、'.join(group[
            (group['当日是否属于热门转债'] == 1)&
            (group['date'] >= row['last_n_date'])& 
            (group['date'] < row['date'])              
        ]['date'].tolist()),
        axis=1
    )
    return group

A_DF['前n天内所有热门的日期'] = A_DF.groupby('bond_code', group_keys=True).apply(filter_dates).reset_index(level=0, drop=True)['前n天内所有热门的日期']
print(A_DF)

                      bond_code        date    cpr       dp       dl    ytm  \
id                                                                            
110043.SH,2023-04-27  110043.SH  2023-04-27  11.78  112.976  124.756  -8.10   
110043.SH,2023-04-28  110043.SH  2023-04-28  12.25  113.247  125.497  -8.43   
110043.SH,2023-05-04  110043.SH  2023-05-04  11.49  114.459  125.949  -9.96   
110043.SH,2023-05-05  110043.SH  2023-05-05  11.91  114.298  126.208  -9.81   
110043.SH,2023-05-08  110043.SH  2023-05-08  11.45  116.600  128.050 -12.42   
...                         ...         ...    ...      ...      ...    ...   
128144.SZ,2023-08-25  128144.SZ  2023-08-25  55.37  115.365  170.735  -0.52   
128144.SZ,2023-08-28  128144.SZ  2023-08-28  53.40  116.004  169.404  -0.68   
128144.SZ,2023-08-29  128144.SZ  2023-08-29  52.88  116.435  169.315  -0.79   
128144.SZ,2023-08-30  128144.SZ  2023-08-30  51.26  116.030  167.290  -0.69   
128144.SZ,2023-08-31  128144.SZ  2023-08-31  50.61  

In [57]:
output_df = DCBS_DF[(DCBS_DF['date']=='2023-08-31') & (DCBS_DF['是否属于过去一段时间内的热门转债'] == 1)].copy()
output_df

,bond_code,date,cpr,dp,dl,ytm,bl,trt,yl,csp,...,cat2,cl,pb,lastday_dp,change,当日是否属于热门转债,过去一段时间是热门转债的天数,是否属于过去一段时间内的热门转债,last_n_date,前n天内所有热门的日期
id,,,,,,,,,,,,,,,,,,,,,
"110058.SH,2023-08-31",110058.SH,2023-08-31,41.76,177.766,219.526,-25.02,2.601,69.13,1.627,5.04,...,NaN,6.32,2.93,180.502,-0.015158,0,13.0,1,2023-06-01,2023-06-05、2023-06-13、2023-06-14、2023-06-16、20...
"110074.SH,2023-08-31",110074.SH,2023-08-31,18.28,139.484,157.764,-6.30,5.093,3.91,2.970,3.57,...,NaN,4.21,1.72,142.398,-0.020464,0,2.0,1,2023-06-01,2023-08-02、2023-08-07
"110091.SH,2023-08-31",110091.SH,2023-08-31,13.55,162.779,176.329,-6.84,20.471,2.28,5.290,14.00,...,NaN,20.07,2.08,164.420,-0.009981,0,3.0,1,2023-06-01,2023-06-12、2023-06-27、2023-07-03
"111008.SH,2023-08-31",111008.SH,2023-08-31,42.65,127.926,170.576,-1.35,3.840,2.83,5.178,46.99,...,NaN,42.14,2.96,126.212,0.013580,0,3.0,1,2023-06-01,2023-06-06、2023-06-09、2023-08-29
"111013.SH,2023-08-31",111013.SH,2023-08-31,104.10,203.650,307.750,-9.20,3.691,120.85,5.523,9.03,...,NaN,9.01,3.13,196.542,0.036165,1,10.0,1,2023-06-01,2023-06-05、2023-06-07、2023-06-21、2023-06-26、20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"128101.SZ,2023-08-31",128101.SZ,2023-08-31,80.87,127.624,208.494,-4.65,2.989,9.73,2.543,13.69,...,NaN,9.66,2.84,128.801,-0.009138,0,2.0,1,2023-06-01,2023-07-04、2023-07-17
"128105.SZ,2023-08-31",128105.SZ,2023-08-31,54.99,108.558,163.548,1.68,7.994,1.99,2.608,7.91,...,NaN,5.54,1.60,109.309,-0.006870,0,2.0,1,2023-06-01,2023-07-07、2023-07-12
"128111.SZ,2023-08-31",128111.SZ,2023-08-31,7.36,375.689,383.049,-33.60,1.099,26.98,2.781,10.63,...,NaN,37.20,2.28,383.598,-0.020618,0,2.0,1,2023-06-01,2023-06-15、2023-07-25


In [58]:
df = A_DF[(A_DF['date']=='2023-08-31') & (A_DF['是否属于过去一段时间内的热门转债'] == 1)].copy()

In [49]:
output_df.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'cn' : '可转债名称',
                                'sc' : '正股代码',
                                'sn' : '正股名称',
                                'dp' : '价格（2点数据计算）',
                                'cpr' : '转股溢价率（两点数据计算）',
                                'dl' : '双低（两点数据计算）',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）'},inplace=True)
output_df.to_excel('结果.xlsx',index=False)

In [108]:
DTBS['A']['123218.SZ']

{'2017-01-03': {'cpr': nan,
  'dp': nan,
  'dl': 999,
  'ytm': nan,
  'bl': nan,
  'trt': nan,
  'yl': nan,
  'csp': nan,
  'csv': nan,
  'ia': 0,
  'qs': 0,
  'qs15': 0,
  'qs30': 0,
  'xx': 0,
  'hs': 0},
 '2017-01-04': {'cpr': nan,
  'dp': nan,
  'dl': 999,
  'ytm': nan,
  'bl': nan,
  'trt': nan,
  'yl': nan,
  'csp': nan,
  'csv': nan,
  'ia': 0,
  'qs': 0,
  'qs15': 0,
  'qs30': 0,
  'xx': 0,
  'hs': 0},
 '2017-01-05': {'cpr': nan,
  'dp': nan,
  'dl': 999,
  'ytm': nan,
  'bl': nan,
  'trt': nan,
  'yl': nan,
  'csp': nan,
  'csv': nan,
  'ia': 0,
  'qs': 0,
  'qs15': 0,
  'qs30': 0,
  'xx': 0,
  'hs': 0},
 '2017-01-06': {'cpr': nan,
  'dp': nan,
  'dl': 999,
  'ytm': nan,
  'bl': nan,
  'trt': nan,
  'yl': nan,
  'csp': nan,
  'csv': nan,
  'ia': 0,
  'qs': 0,
  'qs15': 0,
  'qs30': 0,
  'xx': 0,
  'hs': 0},
 '2017-01-09': {'cpr': nan,
  'dp': nan,
  'dl': 999,
  'ytm': nan,
  'bl': nan,
  'trt': nan,
  'yl': nan,
  'csp': nan,
  'csv': nan,
  'ia': 0,
  'qs': 0,
  'qs15': 0,
 